In [0]:
from datetime import datetime, timedelta 
from getpass import getuser as get_user 
from getpass import getpass as get_password 
from os import environ as enviroment_vars 
from os import setpgrp as set_process_group 
from random import random 
from shlex import split as cmd_split 
from string import punctuation 
from subprocess import Popen, PIPE, STDOUT 


In [0]:
from pyspark import SparkContext, HiveContext, SparkConf  
from pyspark.sql import SparkSession 

ss = SparkSession(sc) 
ss 

In [0]:
#Calculate Parent Lender and Source using DL data
sql = """
select distinct 
t1.*,
t1.parent_level1_lender_cde as PARENT_LENDER_CODE,

case 
when t1.parent_level1_lender_cde in ('40796') then 'Central 1 - BC'
when t1.parent_level1_lender_cde in ('40799') then 'Central 1 - ON'
else t1.parent_level1_lender_nm_en
end as PARENT_LENDER_EN,

t1.parent_level2_lender_nm_en as child_lender,

CASE 
WHEN t1.parent_level1_lender_cde IN 
('40886','40001','40942', '41113', '40798', '40797', '40799', '41023', '40018', '40796', '41879', '41067', '41510', '40011', '41247') 
then 'CREDIT UNIONS'
when t1.parent_level1_lender_cde in ('41042') and t1.lender_cde in ('41042') then 'CREDIT UNIONS'
WHEN t1.parent_level1_lender_cde in ('49020') THEN 'DESJARDINS'
WHEN t1.parent_level1_lender_cde in ('20024', '60001', '60002', '60003', '60004', '60014') then 'KAM_Lenders'
when t1.parent_level1_lender_cde in 
('90032', '30092', '90045', '90040', '90024','90025', '20042', '30006', '30106', '90049', '90044', '30105', '30092') 
then 'MFCs' else 'Other Lenders'
end as Lender_Group,

case 
when t1.parent_level1_lender_cde IN ("40798", "40797", "40018", "40001", "40886") 
THEN "PRARIES"
WHEN t1.parent_level1_lender_cde IN ("41023", "41067", "41113", "40942") 
THEN "ATLANTIC"
WHEN t1.parent_level1_lender_cde IN ("40796", "40011") 
THEN "BRITISH COLUMBIA"
WHEN t1.parent_level1_lender_cde IN ("40799", "41879", "41510", "41247")  
THEN "ONTARIO"
WHEN t1.parent_level1_lender_cde="41042" AND t1.Lender_cde="41042" 
THEN "ONTARIO"
END AS CU_REGION,

case when t1.parent_level1_lender_cde="60002" then 
    (Case
    when lenderchannel="MOSE" and hfs_role is NULL THEN "BROKER" 
    when lenderchannel IN ("BRANCH", "HFLM", "BRM") THEN ""
    ELSE "HFA"
    end) 
end as MOSE_CHANNEL,

CASE WHEN t1.parent_level1_lender_cde="60002" AND lenderchannel="BRM" THEN RU_VP
END AS BRM_RVP,
CASE WHEN t1.parent_level1_lender_cde="60002" AND lenderchannel="HFLM" THEN RU_VP 
END AS HFLM_RD,
CASE WHEN t1.parent_level1_lender_cde="60002" AND (lenderchannel="HFLM" OR HFS_ROLE="HFA") THEN HFS_SUPERVISOR 
END AS SALES_MANAGER,
CASE WHEN t1.parent_level1_lender_cde="60002" AND (lenderchannel="HFLM" OR HFS_ROLE="HFA") THEN HFS_VP 
END AS HFS_RD,

CASE 
    WHEN t1.parent_level1_lender_cde="60002" THEN BUSINESS_SOURCE_EN /*BNS*/
    WHEN t1.parent_level1_lender_cde="60001" THEN
        (CASE
        WHEN transit_desj IN ("00135532", "00103553") THEN "MORTGAGE SPECIALIST"
        ELSE "BRANCH"  END)
        
    WHEN t1.parent_level1_lender_cde="20024" THEN
        (CASE
        WHEN BUSINESS_SOURCE_EN="MORTGAGE SPECIALIST" THEN "MORTGAGE SPECIALIST" ELSE "BRANCH" END) 
    WHEN t1.parent_level1_lender_cde="60003" THEN
        (CASE
        WHEN BUSINESS_SOURCE_EN="MORTGAGE SPECIALIST" THEN "MORTGAGE SPECIALIST" ELSE "BRANCH" END)
    WHEN t1.parent_level1_lender_cde="60004" THEN
        (CASE 
        when transit_desj in ('00427892','00428072','00428472','00428592','00428662','00428672') then "MORTGAGE SPECIALIST"
        when transit_desj in ("00473902", "00491912") then "BROKER" else "BRANCH" END)
    WHEN t1.parent_level1_lender_cde IN ("90040", "90045", "90024", "90025", "20042", "30006","30106", "90049", "90044", "30085", "90043", "90050") THEN "BROKER"
    WHEN t1.parent_level1_lender_cde NOT IN ("60001", "60002", "60004", "49020", "90032") 
    AND BUSINESS_SOURCE_EN not in ("BROKER", "MORTGAGE SPECIALIST", "BRANCH") 
    then "BRANCH" else business_source_en
END AS SOURCE

FROM sb_rmo_ho_reporting.test_lender_dashboard t1 
left join cmwdw_insurance.vwdw_lender_financial_trnst_dim t2
on t1.Lender_cde = t2.lender_cde
where t2.parent_level1_lender_nm_en not in ('CMHC', 'Invalid Lender')
 """

df_lend = ss.sql(sql)
df_lend.createOrReplaceTempView("df_lend")

In [0]:
distinct_count = df_lend.select("parent_level1_lender_nm_en").distinct().count()
print("Distinct count:", distinct_count)

Distinct count: 57


In [0]:
#Calculate BNS Additional HFS RD and CMHC Source
sql_l = """
select 
*,
CASE WHEN df_lend.parent_level1_lender_cde ="60002" AND MOSE_CHANNEL="HFA" THEN HFS_VP 
ELSE HFS_RD
END AS HFS_RD_2,

CASE WHEN SOURCE IN ("COURTIER","BROKER", "RMG", "MCAP", "BRM") THEN "BROKER"
WHEN SOURCE = "MOSE" AND MOSE_CHANNEL="BROKER" THEN "BROKER"
WHEN SOURCE = "MOSE" AND MOSE_CHANNEL="HFA" THEN "MORTGAGE SPECIALIST"
WHEN SOURCE IN ("BRANCH", "SUCCURSALE") THEN "BRANCH"
ELSE "MORTGAGE SPECIALIST" 
END AS CMHC_SOURCE

from df_lend
 """

df_lend2 = ss.sql(sql_l)

In [0]:
df_lend2.write.mode("overwrite").format("orc").saveAsTable("sb_rmo_ho_reporting.test_lender_dashboard2")
#print("Final Base Table with Channel, Source, BNS Contact row count: ", df_fiscal2.count())
print("Final Base Table with Channel, Source, BNS Contact creation completed at: ", str(datetime.now()))

#as of 15.02.2023 BMO is there test_lender_dashboard2

Final Base Table with Channel, Source, BNS Contact creation completed at:  2024-09-06 14:38:48.648332


In [0]:
#print("row count: ", df_lend2.count())